## basic

In [ ]:
from netmiko import Netmiko
connection = Netmiko(host='10.10.1.1', port='22', username='user', password='pass', device_type='cisco_ios')

output = connection.send_command('sh run')
with open('backup', 'w') as f:
    f.write(output)

connection.disconnect()



In [ ]:
from netmiko import ConnectHandler
cisco_device = {
       'device_type': 'cisco_ios',     
       'host': '10.10.1.1',
       'username': 'user',
       'password': 'pass',
       'port': 22,             
       'secret': 'enable-pass',      # this is enable password
       'verbose': True        
       }



connection = ConnectHandler(**cisco_device)
prompt = connection.find_prompt()
print(prompt)
if '>' in prompt:
    connection.enable()

output = connection.send_command('sh run')
print(output)


if not connection.check_config_mode():
    connection.config_mode()

prompt = connection.find_prompt()
print(prompt)

connection.disconnect()




## get Backup and store them into the FTP server

#### solution1: download file locally and them send backup file to the FTP server

In [ ]:
from netmiko import ConnectHandler
from datetime import datetime
import os

# Define the device to connect to
device = {
    'device_type': 'cisco_ios',
    'host': '10.10.1.1',
    'username': 'user',
    'password': 'pass',
    'secret': 'enable-pass',
}

# Define the FTP server details
ftp_server = '10.10.1.3'
ftp_username = 'username'
ftp_password = 'password'
ftp_directory = ''

# Connect to the device
net_connect = ConnectHandler(**device)
prompt = net_connect.find_prompt()
if '>' in prompt:
    net_connect.enable()


# Get the current date and time for the filename
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
backup_filename = f"running-config_{timestamp}.cfg"

# Backup running config to local variable
running_config = net_connect.send_command('show running-config')

# Save the running config to a local file
with open(backup_filename, 'w') as backup_file:
    backup_file.write(running_config)

# Close the connection
net_connect.disconnect()

# Use FTP to transfer the file to the FTP server
from ftplib import FTP

ftp = FTP(ftp_server)
ftp.login(ftp_username, ftp_password)
ftp.cwd(ftp_directory)
with open(backup_filename, 'rb') as file:
    ftp.storbinary(f'STOR {backup_filename}', file)
ftp.quit()

# Optionally, remove the local backup file after transfer
os.remove(backup_filename)

print(f"Backup of running-config completed successfully and uploaded to FTP server as {backup_filename}")


#### solution2: run the command directly on the Cisco router to copy the running configuration to an FTP server without downloading the backup file locally

In [ ]:
from netmiko import ConnectHandler
from datetime import datetime

# Define the device to connect to
device = {
    'device_type': 'cisco_ios',
    'host': '10.10.1.1',
    'username': 'user',
    'password': 'pass',
    'secret': 'enable-pass',
}

# Define the FTP server details
ftp_server = '10.10.1.3'
ftp_username = 'user'
ftp_password = 'pass'


# Connect to the device
net_connect = ConnectHandler(**device)

# Enter enable mode
prompt = net_connect.find_prompt()
if '>' in prompt:
    net_connect.enable()



now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")
backup_filename = f"running-config_{timestamp}.cfg"



# Construct the copy command
copy_command = f"copy running-config ftp://{ftp_username}:{ftp_password}@{ftp_server}/{backup_filename}"

# Send the command to the device
output = net_connect.send_command_timing(copy_command)

# Handle the 'Address or name of remote host []?' prompt
if 'Address or name of remote host' in output:
    output += net_connect.send_command_timing('\n')

# Handle the 'Destination filename [backupfile]?' prompt
if 'Destination filename' in output:
    output += net_connect.send_command_timing('\n')

# Print the output
print(output)

# Close the connection
net_connect.disconnect()

print(f"Backup of running-config completed successfully and uploaded to FTP server as {backup_filename}")


## connect to mikrotik device

In [ ]:
from netmiko import ConnectHandler
mikrotik = {
    'device_type': 'mikrotik_routeros',
    'host':   '10.10.1.3',
    'username': 'admin',
#     'use_keys': True,
#     'key_file': '/Users/laptop/.ssh/id_rsa',
    'password': 'admin',
    'port' : 22          # optional, defaults to 22
    #'secret': 'secret',     # optional, defaults to ''

}
mikrotik_connect=ConnectHandler(**mikrotik)
backup = mikrotik_connect.send_command('export',cmd_verify=False)
with open('backup.rsc', '+a') as f:
    f.write(backup)



## send multiple command

In [ ]:
from netmiko import ConnectHandler
cisco_device = {
       'device_type': 'cisco_ios',     
       'host': '10.10.1.1',
       'username': 'user',
       'password': 'pass',
       'port': 22,             
       'secret': 'pass',      # this is enable password
       'verbose': True        
       }



connection = ConnectHandler(**cisco_device)
prompt = connection.find_prompt()

if '>' in prompt:
    connection.enable()

if not connection.check_config_mode():
    connection.config_mode()

# cmd = ['int gig 2/0', 'ip addr 10.10.57.1 255.255.255.0', 'no sh', 'exit', 
    #    'router ospf 1', 'router-id 1.1.1.1', 'network  10.10.57.1 0.0.0.0']


cmd = ''' 
int gig 2/0
ip addr 10.10.57.1 255.255.255.0
no sh
exit
router ospf 1
router-id 1.1.1.1
network 10.10.57.1 0.0.0.0

'''



cmd = cmd.split('\n')
print(cmd)
output = connection.send_config_set(cmd)
print(output)
connection.disconnect()




## Read config from a file and send command to the router

In [ ]:
from netmiko import ConnectHandler
cisco_device = {
       'device_type': 'cisco_ios',     
       'host': '10.10.1.5',
       'username': 'user',
       'password': 'pass',
       'port': 22,             
       'secret': 'enable-pass',      # this is enable password
       'verbose': True        
       }



connection = ConnectHandler(**cisco_device)
prompt = connection.find_prompt()

if '>' in prompt:
    connection.enable()

if not connection.check_config_mode():
    connection.config_mode()




print(connection.send_config_from_file('./cmd'))

connection.disconnect()




## read config form multiple file

![img](img/3.png)

#### How to read yaml file

In [ ]:
import yaml

# Define the path to the YAML file
yaml_file_path = 'routers.yaml'

# Read the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the list of dictionaries
routers = data['routers']

# Print the list of dictionaries
print(routers)

# Each router's details can be accessed as a dictionary
for router in routers:
    print(f"Host: {router['host']}")
    print(f"Username: {router['username']}")
    print(f"Password: {router['password']}")
    print(f"Port: {router['port']}")
    print("-------------------------------")


### soultion1: send command when asking user to get router configuration file

In [ ]:
import yaml
from netmiko import ConnectHandler

# Define the path to the YAML file
yaml_file_path = 'routers.yaml'

# Read the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the list of dictionaries
routers = data['routers']


for router in routers:
    connection = ConnectHandler(**router)
    prompt = connection.find_prompt()

    if '>' in prompt:
        connection.enable()

    if not connection.check_config_mode():
        connection.config_mode()
    print(f"we are currently connected to this router {router['host']}")
    file_name = input("Enter the configuration file name for this router: ")
    res = connection.send_config_from_file(file_name)
    print(res)
    connection.disconnect()

### sloution2: specifiy the configuration file in the yaml file

In [ ]:
import yaml
from netmiko import ConnectHandler

# Define the path to the YAML file
yaml_file_path = 'routers.yaml'

# Read the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the list of dictionaries
routers = data['routers']


for router in routers:
    config_file_dir = router['config_file_dir']
    del router['config_file_dir']
    print(router)
    connection = ConnectHandler(**router)
    prompt = connection.find_prompt()

    if '>' in prompt:
        connection.enable()

    if not connection.check_config_mode():
        connection.config_mode()
    res = connection.send_config_from_file(config_file_dir)
    print(res)
    connection.disconnect()
    print("-----------------------------------------------------------")

### Get backup from multiple router

In [ ]:
import yaml
from netmiko import ConnectHandler
from ftplib import FTP
from datetime import datetime


# Define the path to the YAML file
yaml_file_path = 'routers.yaml'
# Define the device to connect to


# Define the FTP server details
ftp_server = '10.10.1.3'
ftp_username = 'username'
ftp_password = 'password'
ftp_directory = ''



# Read the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the list of dictionaries
routers = data['routers']


now = datetime.now()
timestamp = now.strftime("%Y%m%d")


for router in routers:
    config_file_dir = router['config_file_dir']
    del router['config_file_dir']
    print(router)
    connection = ConnectHandler(**router)
    prompt = connection.find_prompt()

    if '>' in prompt:
        connection.enable()

    prompt = connection.find_prompt()
    router_name = prompt[:-1]
    backup_filename = f"running-config_{timestamp}_{router_name}.cfg"

    res = connection.send_command('show running-config')


    with open(backup_filename, 'w') as backup_file:
        backup_file.write(res)

    connection.disconnect()


    ftp = FTP(ftp_server)
    ftp.login(ftp_username, ftp_password)
    ftp.cwd(ftp_directory)
    with open(backup_filename, 'rb') as file:
        ftp.storbinary(f'STOR {backup_filename}', file)
    ftp.quit()

    # Optionally, remove the local backup file after transfer
    # os.remove(backup_filename)

    print(f"Backup of running-config completed successfully and uploaded to FTP server as {backup_filename}")
    print("----------------------------------------------------------------------------------------")

## Use multithreading

In [ ]:
import yaml
from netmiko import ConnectHandler
from ftplib import FTP
from datetime import datetime
import threading

def get_backup(router, ftp_server, ftp_username, ftp_password, ftp_directory):
    connection = ConnectHandler(**router)
    prompt = connection.find_prompt()

    if '>' in prompt:
        connection.enable()

    prompt = connection.find_prompt()
    router_name = prompt[:-1]
    now = datetime.now()
    timestamp = now.strftime("%Y%m%d")
    backup_filename = f"running-config_{timestamp}_{router_name}.cfg"

    res = connection.send_command('show running-config')


    with open(backup_filename, 'w') as backup_file:
        backup_file.write(res)

    connection.disconnect()


    ftp = FTP(ftp_server)
    ftp.login(ftp_username, ftp_password)
    ftp.cwd(ftp_directory)
    with open(backup_filename, 'rb') as file:
        ftp.storbinary(f'STOR {backup_filename}', file)
    ftp.quit()



# Define the path to the YAML file
yaml_file_path = 'routers.yaml'
# Define the device to connect to


# Define the FTP server details




# Read the YAML file
with open(yaml_file_path, 'r') as file:
    data = yaml.safe_load(file)

# Access the list of dictionaries
routers = data['routers']


threads = []
for router in routers:
    config_file_dir = router['config_file_dir']
    del router['config_file_dir']
    # print(router)
    # get_backup(router, ftp_username='user', ftp_password='pass', ftp_server='10.10.1.3', ftp_directory='')
    th = threading.Thread(target=get_backup, args=(router, '10.10.1.3', 'user', 'pass', ''))
    threads.append(th)

for th in threads:
    th.start()

for th in threads:
    th.join()